<a href="https://colab.research.google.com/github/OneFineStarstuff/Onefinebot/blob/main/Example_CNN_for_Galaxy_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

# Placeholder for loading your dataset
# Replace this with your actual data loading code
train_images = torch.randn(100, 3, 64, 64)  # Example: 100 images, 3 channels, 64x64 resolution
train_labels = torch.randint(0, 5, (100,))  # Example: 100 labels, 5 classes

# Custom Dataset class
class GalaxyDataset(Dataset):
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

# Simple CNN architecture
class GalaxyCNN(nn.Module):
    def __init__(self):
        super(GalaxyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)

        # Dummy input to determine the size of the flattened layer
        self._dummy_input = torch.randn(1, 3, 64, 64)
        flattened_size = self._get_flattened_size()

        self.fc1 = nn.Linear(flattened_size, 128)
        self.fc2 = nn.Linear(128, 5)  # Assuming 5 classes of galaxy morphologies

    def _get_flattened_size(self):
        x = self.pool(torch.relu(self.conv1(self._dummy_input)))
        x = self.pool(torch.relu(self.conv2(x)))
        return x.view(1, -1).size(1)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Create dataset and dataloader
train_dataset = GalaxyDataset(train_images, train_labels)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Initialize model, loss function, and optimizer
model = GalaxyCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")